### This notebook is used to communicate with Unity VirtualHome over a socket
### Each frame from Unity is sent directly to this Python Notebook where it's processed before being passed into a trained YOLACT Model for Inference in Real-Time. 
### The YOLACT Model is trained on a synthetic home indoor dataset

Author: Adam Goldstein



In [1]:
# Imports

import numpy as np
import urllib
import time
import cv2
from yolact_edge.inference import YOLACTEdgeInference
import zmq
import struct
import gzip


In [2]:
# Load Model

# weights = "yolact_edge_resnet50_54_800000.pth"
weights = "yolact_edge/weights/yolact_base_26666_1280000.pth"
# All available model configs, depends on which weights

In [3]:
# Load YOLACT Configuration

model_configs = [
    'yolact_base_config',
    'yolact_edge_config',
    'yolact_edge_mobilenetv2_config',
    'yolact_edge_vid_config',
    'yolact_edge_vid_minimal_config',
    'yolact_edge_vid_trainflow_config',
    'yolact_edge_youtubevis_config',
    'yolact_resnet50_config',
    'yolact_resnet152_config',
    'yolact_edge_resnet50_config',
    'yolact_edge_vid_resnet50_config',
    'yolact_edge_vid_trainflow_resnet50_config',
    'yolact_edge_youtubevis_resnet50_config',
]
config = model_configs[1]

# Load dataset
datasets = [
    'SyntheticHome',
    'coco2014_dataset',
    'coco2017_dataset',
    'coco2017_testdev_dataset',
    'flying_chairs_dataset',
    'youtube_vis_dataset',
]
dataset = datasets[0]
# Used tensorrt calibration
calib_images = "./data/calib_images"
# Override some default configuration
# config_ovr = {
#     'use_fast_nms': True,  # Does not work with regular nms
#     'mask_proto_debug': False
# }
config_ovr = {
    '--fast_nms': True,  # Does not work with regular nms
    'mask_proto_debug': False
}

# Load Model
model_inference = YOLACTEdgeInference(
    weights, config, dataset, calib_images, config_ovr)

Configuring YOLACT edge...
Loading YOLACT edge model...
Model ready for inference...


In [4]:
context = zmq.Context()
socket = context.socket(zmq.SUB)

# Subscribe to all messages
socket.connect("tcp://localhost:5555")
socket.setsockopt(zmq.SUBSCRIBE, b"")

# print("Waiting for messages...")

print('Starting up on {} port {}'.format('localhost', 5555))

print("Waiting for messages...")

payload_size = struct.calcsize("Q")

data = b""

Starting up on localhost port 5555
Waiting for messages...


In [5]:
print("Benchmarking performance...")
start = time.time()
prev_time = 0

while True:
    t1 = time.time()
    try:
        while len(data) < payload_size:
            packet = socket.recv()

            if not packet: break
            data+=packet
        
        image_data = gzip.decompress(data)

        img = np.frombuffer(image_data, dtype=np.uint8)

        img = img.reshape((480, 640, 4))
        img = cv2.resize(img, (1280, 960))

        # img = img.reshape((320, 512, 4))
        # img = cv2.resize(img, (1024, 640))

        # # Flip image
        img = cv2.flip(img, 0)
        # # Convert to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

        # img = cv2.convertScaleAbs(img, alpha=3.5, beta=5)

        # Time inference

        # t3 = time.time()
        p = model_inference.predict(img, False)
        # t4 = time.time()
        # print(f"Time taken: {t4 - t3} seconds")

        

        print(f"Average {1 / (time.time() - prev_time)} FPS")

        if p:
           
            cv2.imshow("Unity Cam", p['img'])
            # cv2.resizeWindow("Garnet Cam", 1280, 960)
            # Change window size
            
            cv2.waitKey(1)
        else:
            print("No prediction")

        prev_time = time.time()
        data = b""
        t2 = time.time()
        # print(f"Time taken: {t2 - t1} seconds")


    except KeyboardInterrupt:
        print("Shutting down...")
        break


Benchmarking performance...
Average 5.959942960873655e-10 FPS


QObject::moveToThread: Current thread (0x555d30085300) is not the object's thread (0x555d30374110).
Cannot move to target thread (0x555d30085300)

QObject::moveToThread: Current thread (0x555d30085300) is not the object's thread (0x555d30374110).
Cannot move to target thread (0x555d30085300)

QObject::moveToThread: Current thread (0x555d30085300) is not the object's thread (0x555d30374110).
Cannot move to target thread (0x555d30085300)

QObject::moveToThread: Current thread (0x555d30085300) is not the object's thread (0x555d30374110).
Cannot move to target thread (0x555d30085300)

QObject::moveToThread: Current thread (0x555d30085300) is not the object's thread (0x555d30374110).
Cannot move to target thread (0x555d30085300)

QObject::moveToThread: Current thread (0x555d30085300) is not the object's thread (0x555d30374110).
Cannot move to target thread (0x555d30085300)

QObject::moveToThread: Current thread (0x555d30085300) is not the object's thread (0x555d30374110).
Cannot move to tar

Average 1.8157293798266048 FPS
Average 17.903488664933647 FPS
Average 23.39148620537731 FPS
Average 6.442398167874209 FPS
Average 3.9559612883388713 FPS
No predictions!
Average 23.6024895051377 FPS
No prediction
Average 3.3480694119271717 FPS
Average 16.628820406690693 FPS
Average 18.401477627197465 FPS
Average 4.118632367511707 FPS
Average 21.42804448781285 FPS
Average 19.294446693408897 FPS
No predictions!
Average 17.391483186134263 FPS
No prediction
Average 18.46767290723683 FPS
Average 5.179272064952304 FPS
Average 23.448502568861706 FPS
Average 18.710955866936114 FPS
Average 18.12553856259426 FPS
Average 21.975123908920395 FPS
Average 21.745891185101463 FPS
Average 21.871875765903415 FPS
Average 21.41283860362062 FPS
Average 18.333751502568024 FPS
Average 20.449645058116857 FPS
Average 19.97068892454636 FPS
Average 16.99467992431149 FPS
Average 20.484400945515638 FPS
Average 17.636909517522096 FPS
Average 19.48039329521757 FPS
Average 18.78814918339739 FPS
Average 18.6032351493162

In [6]:
# close the socket
socket.close()